In [ ]:
!pip install tensorflow_recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.1 MB/s eta 0:00:00


In [ ]:
import string
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_recommenders as tfrs
import json
from collections import Counter
from typing import Dict, Text
from ast import literal_eval
from datetime import datetime
from wordcloud import WordCloud
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings('ignore')

In [29]:
import requests

rating = requests.get('http://34.101.237.85:3000/rating')
fiction_metadata = requests.get('http://34.101.237.85:3000/fiction')

In [40]:
rating_data = rating.json()
fiction_data = fiction_metadata.json()

df = pd.DataFrame(rating_data["data"])
df1 = df.drop(df.index[0])

f_df = pd.DataFrame(fiction_data["data"])
f_df1 = f_df.drop(df.index[0])

print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")

df1.head(10)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


,user_id,fiction_id,click,like,rating
1,55,25,1,1,2
2,7,15,1,1,2
3,46,16,1,1,1
4,44,15,1,1,2
5,72,17,1,1,5
6,92,25,1,1,3
7,85,19,1,1,1
8,48,10,1,1,3
9,32,21,1,1,3
10,74,3,1,1,1


In [44]:
fiction = f_df1[['fiction_id','title','overview','language','tags','genres','chapter']]
fiction.head(10)

,fiction_id,title,overview,language,tags,genres,chapter
1,1,Kejahatan Cinta,"Seorang detektif swasta, Anya, sedang menyelid...",Indonesia,"detektif,pembunuhan,misteri,cinta,romansa","romansa,misteri",2
2,2,Raja Terakhir,"Di dunia yang dikuasai oleh iblis, seorang pem...",Indonesia,"fantasi,aksi,petualangan,pertarungan,kekuatan","fantasi,aksi",1
3,3,Perjalanan ke Masa Depan,Seorang ilmuwan muda menemukan cara untuk mela...,Indonesia,"fiksi ilmiah,petualangan,perjalanan waktu,mist...","sci-fi,petualangan",1
4,4,Cinta di Balik Tirai,Dua orang yang berasal dari latar belakang yan...,Indonesia,"romansa,drama,perbedaan latar belakang,rahasia...","romansa,drama",1
5,5,Pewaris Sang Dewa,Seorang pemuda yang terpilih sebagai pewaris s...,Indonesia,"fantasi,aksi,petualangan,dewa,takdir","fantasi,aksi",1
6,6,Legenda Tanah Jawa,Seorang pemuda yang hidup di masa kini menemuk...,Indonesia,"antasi,aksi,petualangan,sejarah,legenda","sci-fi,petualangan",1
7,7,Reinkarnasi,Seorang wanita yang meninggal dalam kecelakaan...,Indonesia,"reinkarnasi,keluarga,cinta,kehilangan","fantasi,drama",1
8,8,Penyelamat Dunia,Seorang remaja yang memiliki kekuatan super ha...,Indonesia,"sci-fi,aksi,petualangan,alien, penyelamatan dunia","sci-fi,petualangan",1
9,9,Cinta Tak Terduga,Dua orang yang awalnya tidak menyukai satu sam...,Indonesia,"romansa,komedi,persahabatan,sekolah","romansa,komedi",1
10,10,Kisah Cinta yang Tak Terlupakan,Sepasang kekasih yang harus terpisah karena su...,Indonesia,"romansa,drama,cinta pertama,reuni","romansa,drama",1


In [48]:
df1['user_id'] = df1['user_id'].astype(int)
df1['fiction_id'] = df1['fiction_id'].astype(int)
df1['click'] = df1['click'].astype(int)
df1['like'] = df1['like'].astype(int)
df1['rating'] = df1['rating'].astype(int)

fiction['fiction_id'] = fiction['fiction_id'].astype(int)
fiction['overview'] = fiction['overview'].astype(str)
fiction['language'] = fiction['language'].astype(str)
fiction['tags'] = fiction['tags'].astype(str)
fiction['genres'] = fiction['genres'].astype(str)
fiction['chapter'] = fiction['chapter'].astype(str)

In [49]:
from scipy.sparse import coo_matrix

#menginisiasi ratings_mat_coo dengan sparse matrix dari kolom rating, user_index dan book_index
ratings_mat_coo = coo_matrix((df1["rating"], (df1["user_id"], df1["fiction_id"])))

In [50]:
#mengetahui ukuran dari matrix
ratings_mat_coo.shape

(101, 26)

In [51]:
#menginversikan coo matrix ke csr matrix
ratings_mat = ratings_mat_coo.tocsr()

In [52]:
df1[df1["user_id"] == 1]

,user_id,fiction_id,click,like,rating
16,1,20,1,1,1
32,1,16,1,1,2
135,1,23,1,1,5
319,1,1,1,1,3
368,1,16,1,1,3
730,1,6,1,1,3
882,1,17,1,1,2


In [54]:
my_index = 1

#menghitung similarity menggunakan cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(ratings_mat[my_index,:], ratings_mat).flatten()

In [55]:
similarity[2]

0.06919444144937047